# Binary classification with *k*-nearest neighbours on Gaussian data

This notebook illustrates some of the basic principles of supervised machine learning using a k-nearest neighbours approach. The basic output is a video file displaying how the decision boundary classifying two 2D point clusters changes with *k*, and how the classification performance changes from underfitting large *k* values to overfitting with small *k*, passing nicely through an accuracy maximum (error minimum) somewhere in between. This bias-variance tradeoff to maximise real-world performance is fundamental to statistical model building. 

This notebook absolutely does not capture the challenges of real-world model building, including, but not limited, selection bias, non-parametric distributions (as opposed to nice, analytic Gaussians), data sparsity, having many (hyper)parameters rather than just one (*k*), etc.


A video summarising the output of the noteboook is available here: https://www.youtube.com/watch?v=bmDdo_5IP2k



In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib.colors import ListedColormap
from sklearn import model_selection, neighbors
from matplotlib.ticker import FormatStrFormatter
import random

#plt.rcParams['pcolor.shading']
pd.options.mode.chained_assignment = None
style.use('ggplot')

## 1. Generate data 

Firstly, generate data points from two bivariate Gaussian distributions. For example, height and weight for adult men and women.

In [4]:
# Pick Gaussian distribution parameters and number of data points
n_0 = 1000                         # number of data points in class 0
n_1 = 1000                         # number of data points in class 1
mean_0 = [0.5, 3]                  # mean of class 0 data
cov_0 = [[1.5, 0.0], [0.0, 1.5]]   # covariance matrix of class 0 data
mean_1 = [0, 0]                    # mean of class 1 data
cov_1 = [[1.2, 0.0], [0.0, 1.2]]   # covariance matrix of class 1 data

# Train / test split
test_set_size = 0.5                # % of data held out for test set

# Create data points and place into dataframes with class label column
x1_0, x2_0 = np.random.multivariate_normal(mean_0, cov_0, n_0).T
x1_1, x2_1 = np.random.multivariate_normal(mean_1, cov_1, n_1).T

c0 = pd.DataFrame(np.transpose([x1_0, x2_0]), columns=['x1', 'x2'])
c0['class'] = 0
c1 = pd.DataFrame(np.transpose([x1_1, x2_1]), columns=['x1', 'x2'])
c1['class'] = 1

# Concatenate into single dataframe
df = pd.concat([c1, c0]).reset_index()

# Assign training / test labels
df['dataset'] = random.choices(['train', 'test'],
                               weights = [1-test_set_size, test_set_size],
                               k = n_0+n_1)

# Resolve data into training/test sets
X_trn = df[(df['dataset'] == 'train')][['x1', 'x2']]
X_tst = df[(df['dataset'] == 'test')][['x1', 'x2']]
y_trn = df[(df['dataset'] == 'train')]['class']
y_tst = df[(df['dataset'] == 'test')]['class']

# Resolve data into training/test sets, further resolved into the different target values
X_trn_1 = df[(df['dataset'] == 'train') & (df['class'] == 1)][['x1', 'x2']].values
X_trn_0 = df[(df['dataset'] == 'train') & (df['class'] == 0)][['x1', 'x2']].values
X_tst_1 = df[(df['dataset'] == 'test')  & (df['class'] == 1)][['x1', 'x2']].values
X_tst_0 = df[(df['dataset'] == 'test')  & (df['class'] == 0)][['x1', 'x2']].values
y_trn_1 = df[(df['dataset'] == 'train') & (df['class'] == 1)]['class'].values
y_trn_0 = df[(df['dataset'] == 'train') & (df['class'] == 0)]['class'].values
y_tst_1 = df[(df['dataset'] == 'test')  & (df['class'] == 1)]['class'].values
y_tst_0 = df[(df['dataset'] == 'test')  & (df['class'] == 0)]['class'].values

## 2. Prepare background grid in the feature space for plotting the prediction regions

In [9]:
# Background grid fineness. Reduce to about 100 for a *significant* speedup
grid_resolution = 300

# Form a regular 2D grid spanning the entire feature space
x1_grid = np.linspace(min(df['x1']), max(df['x1']), grid_resolution)
x2_grid = np.linspace(min(df['x2']), max(df['x2']), grid_resolution)
ee, nn = np.meshgrid(x1_grid, x2_grid)

# Flatten meshgrid to create two-column array of all coordinate pairs
prediction_grid = np.vstack([np.ravel(nn), np.ravel(ee)]).T
prediction_grid = np.flip(prediction_grid, axis=1)

## 3. Perform successive predictions over range of *k* values and output images

First create some functions to i) obtain the confusion matrix for a given value of *k*, and ii) to create and save a figure displaying everything that's happening.

In [6]:
def get_confusion_matrix_elements(x_set_1, x_set_0, y_set_1, y_set_0, classifier):
    
    """
    Returns the elements of the confusion matrix from a binary classifier applied to 
    an array of input data (x) and output classes (y)
    """
    
    y_set_1_pred = classifier.predict(x_set_1)
    y_set_0_pred = classifier.predict(x_set_0)

    tp_set = y_set_1_pred == y_set_1
    fn_set = y_set_1_pred != y_set_1
    tn_set = y_set_0_pred == y_set_0
    fp_set = y_set_0_pred != y_set_0

    x_set_tp = x_set_1[tp_set]
    x_set_fn = x_set_1[fn_set]
    x_set_tn = x_set_0[tn_set]
    x_set_fp = x_set_0[fp_set]

    return x_set_tp, x_set_fn, x_set_tn, x_set_fp

def save_image(df_,
               k_max_,
               ee_,
               nn_,
               decision_regions_,
               X_trn_tp_, 
               X_trn_fn_, 
               X_trn_tn_, 
               X_trn_fp_, 
               X_tst_tp_, 
               X_tst_fn_, 
               X_tst_tn_, 
               X_tst_fp_, 
               one_over_k_,
               test_error_rate_,
               train_error_rate_,
               sensitivity_tst_,
               specificity_tst_,
               sensitivity_trn_,
               specificity_trn_,
               k_, 
              ):
    
    """
    Plots a four-panel image summarising the training data, test data, classificaion
    regions, and classification metrics.
    """
    
    # Initialise plotting figure
    fig = plt.figure(figsize=(1920/100,  1080/100))
    custom_colors = ListedColormap(['#A1E1FF', '#FFC8C6'])

    ax1 = plt.subplot2grid((2, 5), (0,0), colspan=2, rowspan=2)
    ax2 = plt.subplot2grid((2, 5), (0,3), colspan=2, rowspan=2)
    ax3 = plt.subplot2grid((2, 5), (0,2), colspan=1, rowspan=1)
    ax4 = plt.subplot2grid((2, 5), (1,2), colspan=1, rowspan=1)

    ax1.set_xlabel(r'$x_1$')
    ax1.set_ylabel(r'$x_2$')
    ax1.set_xlim([min(df_['x1']), max(df_['x1'])])
    ax1.set_ylim([min(df_['x2']), max(df_['x2'])])

    ax2.set_xlabel(r'$x_1$')
    ax2.set_ylabel(r'$x_2$')
    ax2.set_xlim([min(df_['x1']), max(df_['x1'])])
    ax2.set_ylim([min(df_['x2']), max(df_['x2'])])

    ax3.set_xlabel(r'1/k')
    ax3.set_ylabel('error')
    ax3.set_xscale('log')
    ax3.set_xlim([1/k_max_, 1])
    ax3.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax3.set_title('Error rates vs. 1/k')

    ax4.set_xlabel(r'1/k')
    ax4.set_ylabel('sensitivity / specificity')
    ax4.set_xscale('log')
    ax4.set_xlim([1/k_max_, 1])
    ax4.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
    ax4.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax4.set_title('Sens./Spec. vs. 1/k')

    # plot prediction regions
    ax1.pcolor(ee_, nn_,  decision_regions_, cmap=custom_colors, shading='auto')
    ax2.pcolor(ee_, nn_,  decision_regions_, cmap=custom_colors, shading='auto')

    # plot the training data scatter plot
    ax1.scatter(X_trn_tp_[:, 0], X_trn_tp_[:, 1], s=50, marker='.', c='red', label='true positives ({0} set)'.format('training'))
    ax1.scatter(X_trn_fn_[:, 0], X_trn_fn_[:, 1], s=50, marker='x', c='red', label='false negatives ({0} set)'.format('training'))
    ax1.scatter(X_trn_tn_[:, 0], X_trn_tn_[:, 1], s=50, marker='.', c='blue', label='true negatives ({0} set)'.format('training'))
    ax1.scatter(X_trn_fp_[:, 0], X_trn_fp_[:, 1], s=50, marker='x', c='blue', label='false positives ({0} set)'.format('training'))
    ax1.set_title('{0}-nearest neighbours binary classifier: training set'.format(k), fontsize=14)

    # plot the test data scatter plot
    ax2.scatter(X_tst_tp_[:, 0], X_tst_tp_[:, 1], s=50, marker='.', c='red', label='true positives ({0} set)'.format('test'))
    ax2.scatter(X_tst_fn_[:, 0], X_tst_fn_[:, 1], s=50, marker='x', c='red', label='false negatives ({0} set)'.format('test'))
    ax2.scatter(X_tst_tn_[:, 0], X_tst_tn_[:, 1], s=50, marker='.', c='blue', label='true negatives ({0} set)'.format('test'))
    ax2.scatter(X_tst_fp_[:, 0], X_tst_fp_[:, 1], s=50, marker='x', c='blue', label='false positives ({0} set)'.format('test'))
    ax2.set_title('{0}-nearest neighbours binary classifier: test set'.format(k), fontsize=14)

    # plot test error array against 1/k array
    ax3.plot(one_over_k_, test_error_rate_, color='black', lw='2', label='test error')
    ax3.plot(one_over_k_, train_error_rate_, color='red', lw='2', label='training error')

    # plot sensitivity and specificity arrays against 1/k array
    ax4.plot(one_over_k_, sensitivity_tst_, c='black', lw='2', label='sensitivity (test set)')
    ax4.plot(one_over_k_, specificity_tst_, c='green', lw='2', label='specificity (test set)')
    ax4.plot(one_over_k_, sensitivity_trn_, c='red', lw='2', label='sensitivity (training set)')
    ax4.plot(one_over_k_, specificity_trn_, c='fuchsia', lw='2', label='specificity (training set)')

    plt.tight_layout()

    ax1.legend(loc='upper right')
    ax2.legend(loc='upper right')
    ax3.legend(loc='lower left')
    ax4.legend(loc='lower right')

    plt.savefig('images/{0}_knn.png'.format(k_))
    plt.close(fig)

Next, create a directory for the images to be saved to.

In [7]:
!mkdir -p images

Loop through some values of *k*, and generate the image displaying the model prediction regions and decision boundary overlaid onto the training and test sets, along with two graphs showing the error (1 - accuracy), specificity (how often the model gets it right for the blue points), and sensitivity (how often it gets it right for the red points). Save these images to the directory just created.

In [8]:
# empty arrays to be populated in the loop for error vs. 1/k plot
one_over_k = np.empty(0)
test_error_rate = np.empty(0)
train_error_rate = np.empty(0)
sensitivity_tst = np.empty(0)
specificity_tst = np.empty(0)
sensitivity_trn = np.empty(0)
specificity_trn = np.empty(0)

# Loop variables
k_max = 501                        # max number of neighbours considered
k_min = 1                          # min number of neighbours considered
step = 2                           # step between k values

# loop to produce successive images over k range
for k in range(k_max, k_min-1, -step):

    # Fit the training data
    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(X_trn, y_trn)

    # Pass the coordinate pairs to the classifier to return class predictions for each point
    decision_regions = clf.predict(prediction_grid)

    # Repackage the class predictions into the shape corresponding to the meshgrid
    decision_regions = decision_regions.reshape(ee.shape)

    # classify training and test sets corresponding to entries of confusion matrix
    X_trn_tp, X_trn_fn, X_trn_tn, X_trn_fp = \
    get_confusion_matrix_elements(X_trn_1, X_trn_0, y_trn_1, y_trn_0, clf)
    
    X_tst_tp, X_tst_fn, X_tst_tn, X_tst_fp = \
    get_confusion_matrix_elements(X_tst_1, X_tst_0, y_tst_1, y_tst_0, clf)

    # count number of true positives, false positives, true negatives and false negatives
    tst_tp, tst_fn, tst_tn, tst_fp = len(X_tst_tp), len(X_tst_fn), len(X_tst_tn), len(X_tst_fp)
    trn_tp, trn_fn, trn_tn, trn_fp = len(X_trn_tp), len(X_trn_fn), len(X_trn_tn), len(X_trn_fp)

    # form arrays of error rates (updates with each kth loop iteration)
    test_error_rate = \
    np.append(test_error_rate, np.array([(tst_fn + tst_fp) / (tst_tp + tst_fn + tst_tn + tst_fp)]))
    
    train_error_rate = \
    np.append(train_error_rate, np.array([(trn_fn + trn_fp) / (trn_tp + trn_fn + trn_tn + trn_fp)]))

    # array of 1/k values for plotting test and training error
    one_over_k = np.append(one_over_k, np.array([1/k]))

    sensitivity_tst = np.append(sensitivity_tst, np.array([tst_tp / (tst_tp + tst_fn)]))
    specificity_tst = np.append(specificity_tst, np.array([tst_tn / (tst_tn + tst_fp)]))
    sensitivity_trn = np.append(sensitivity_trn, np.array([trn_tp / (trn_tp + trn_fn)]))
    specificity_trn = np.append(specificity_trn, np.array([trn_tn / (trn_tn + trn_fp)]))

    save_image(df_=df,
               k_max_=k_max,
               ee_=ee,
               nn_=nn,
               decision_regions_=decision_regions,   
               X_trn_tp_=X_trn_tp, 
               X_trn_fn_=X_trn_fn, 
               X_trn_tn_=X_trn_tn, 
               X_trn_fp_=X_trn_fp, 
               X_tst_tp_=X_tst_tp, 
               X_tst_fn_=X_tst_fn, 
               X_tst_tn_=X_tst_tn, 
               X_tst_fp_=X_tst_fp, 
               one_over_k_=one_over_k,
               test_error_rate_=test_error_rate,
               train_error_rate_=train_error_rate,
               sensitivity_tst_=sensitivity_tst,
               specificity_tst_=specificity_tst,
               sensitivity_trn_=sensitivity_trn,
               specificity_trn_=specificity_trn,
               k_=k,
              )


In [2]:
# create video from images
!ffmpeg -y -framerate 30 -pattern_type glob -i 'images/*.png' video.mp4

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.0 (crosstool-NG 1.24.0.133_b0863d8_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --pkg-config=/home/conda/feedstock_root/build_artifacts/ffmpeg_1609680890771/_build_env/bin/pkg-config
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostpr